In [1]:
import subprocess
import time
import os
from my_MRI_tool_kit import DirectoryReader_dict
from tqdm import tqdm

# ███████████████
"""shell命令运行"""
# ███████████████
def run_command(command, message=''):
    start_time = time.time()
    if message!='':
        print(f'【{message}】',end="")
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    end_time = time.time()
    duration = end_time - start_time
    
    # 检查命令的返回值
    if result.returncode == 0:
        print(f'命令执行完毕,耗时{duration:.2f}秒')
    else:
        print(f"命令执行失败,耗时{duration:.2f}秒")


# ███████████████
"""找到最适合打头阵的序列，返回两个列表"""
# ███████████████
def find_which_first(all_files, priority):
    all_files = list(set(all_files) & set(priority))
    # priority = ['T1CE.nii.gz', 'T1.nii.gz', 'T2.nii.gz', 'T2Flair.nii.gz', 'fDWI_2.nii.gz', 'ADC.nii.gz', 'ADC_manual.nii.gz']
    
    # 按顺序找到最好的序列
    def select_target_file(files_list):
        for file in priority:
            if file in files_list:
                return file
        return None
    
    target_file = ''
    target_file = select_target_file(all_files)
    remaining_files =  []
    remaining_files = [file for file in all_files if file != target_file]

    print("先配准:", target_file)
    print("后配准:", remaining_files)
    
    # target_file是一个文件，里面必然有东西且只有一个；remaining_files是文件的列表，可以为空列表
    return [target_file,remaining_files]   


# ███████████████
"""单个MRI的刚体配准"""
# ███████████████
def rigid_regisitration(template_file, input_file, output_file):
    # 相对路径
    work_dir = os.path.dirname(input_file)
    
    # 放射变换文件的前缀
    mat_prefix = os.path.join(work_dir,'rigidTemp_')
    mat_name = mat_prefix+'0GenericAffine.mat'
    
    # 一阶段
    command_Rigid = ' '.join([
        antsRegistration,'-d 3',
        '-m',f'MI[{template_file},{input_file},1,32,Regular,0.25]','-w [0.005, 0.995]','--float','-v',
        '-t','Rigid[0.1]',
        '-o',mat_prefix,
        '-c [1000x500x250x100x0,1e-6,10]','-s 4x3x2x1x1vox','-f 12x8x4x2x1',
    ])

    command_trans =  ' '.join([
        antsApplyTransforms,'-d 3','-n','Linear','-v',
        '-i',input_file,
        '-r',template_file,
        '-t',mat_name,
        '-o',output_file,
    ])

    run_command(command_Rigid,f'Rigid: {input_file}→{template_file}\n→{output_file}')
    run_command(command_trans,'')
    
    # 删除临时的变换文件
    os.remove(mat_name)

    
# ███████████████
"""SyN配准，生成3个仿射变换文件，和一个最终文件"""
# ███████████████
def SyN_regisitration(template, input_file):   
    # 保存文件的文件夹
    dir_path = os.path.dirname(input_file)
    fileName = os.path.basename(input_file)
    
    output_file = os.path.join(dir_path,'______regd_'+fileName[2:])
    print(f'【拟生成文件】{output_file}')
    
#     # ━━━━━━━━━━━━━━━━━━━━━━━━━
#     # ━━━━━━━━━━━━━━━━━━━━━━━━━
#     # 一阶段
#     mat_rigid_prefix = os.path.join(dir_path,'____Rigid_')
#     mat_rigid_name = mat_rigid_prefix+'0GenericAffine.mat'
    
#     command_Rigid = ' '.join([
#         antsRegistration,'-d 3', '-w [0.005, 0.995]','--float','-v',
#         '-m',f'MI[{template},{input_file},1,32,Regular,0.25]',
#         '-t','Rigid[0.1]',
#         '-o',mat_rigid_prefix,
#         '-c [1000x500x250x100x0,1e-6,10]','-s 4x3x2x1x1vox','-f 12x8x4x2x1',
#     ])

#     command_trans =  ' '.join([
#         antsApplyTransforms,'-d 3','-n','Linear','-v',
#         '-i',input_file,    # 这时候首先输入的是最开始的文件
#         '-r',template,   
#         '-t',mat_rigid_name,
#         '-o',output_file,
#     ])
    
#     run_command(command_Rigid,f'Rigid刚体配准(在{input_file} 上做)')
#     run_command(command_trans,f'应用rigid配准')
    
#     # 如果成功执行第一步，那么意味着output_file已经生成，那么下一步输入的文件变成原来的output_file
#     input_file = output_file

    # ━━━━━━━━━━━━━━━━━━━━━━━━━
    # ━━━━━━━━━━━━━━━━━━━━━━━━━
    # 二阶段
    mat_affine_prefix = os.path.join(dir_path,'____Affine_')
    mat_affine_name = mat_affine_prefix+'0GenericAffine.mat'
    
    command_Affine = ' '.join([
        antsRegistration,'-d 3', '-w [0.005, 0.995]','--float','-v',
        '-m',f'MI[{template},{input_file},1,32,Regular,0.25]',
        '-t','Affine[0.1]',
        '-o',mat_affine_prefix,
        '-c [1000x500x250x100x0,1e-6,10]','-s 4x3x2x1x1vox','-f 12x8x4x2x1',
    ])

    command_trans =  ' '.join([
        antsApplyTransforms,'-d 3','-n','Linear','-v',
        '-i',input_file,
        '-r',template,
        '-t',mat_affine_name,
        '-o',output_file,
    ])

    run_command(command_Affine,f'Affine放射变换')
    run_command(command_trans,f'应用affine变换')

    # ━━━━━━━━━━━━━━━━━━━━━━━━━
    # ━━━━━━━━━━━━━━━━━━━━━━━━━
    # 三阶段
    mat_SyN_prefix = os.path.join(dir_path,'____SyN_')
    mat_SyN_name = mat_SyN_prefix+'0Warp.nii.gz'

    command_SyN = ' '.join([
        antsRegistration,'-d 3', '-w [0.005, 0.995]','--float','-v',
        '-m',f'MI[{template},{output_file},1,32,Regular,0.25]',
        '-t','SyN[0.1,3,0]',
        '-o',mat_SyN_prefix,
        '-c [100x100x70x50x0,1e-6,10]','-s 5x3x2x1x0vox','-f 10x6x4x2x1',
    ])

    command_trans =  ' '.join([
        antsApplyTransforms,'-d 3','-n','Linear','-v',
        '-i',output_file,
        '-r',template,
        '-t',mat_SyN_name,
        '-o',output_file,
    ])

    run_command(command_SyN,f'SyN配准变换')
    run_command(command_trans,f'应用syn配准')

    
# ███████████████
# 所有互相配准后的文件应用变换到MNI152
# ███████████████
def apply_all_transforms(template, input_file_list):
    # 工作路径
    work_dir = os.path.dirname(input_file_list[0])
    
    # 仿射变换的文件,因为所有的其他序列都做了组内配准，
    # 而组内配准之前，T1CE已经和MNI152完成了刚体配准，所以现在所有的序列只需要再做一次Affnie和SyN即可
    Affine = os.path.join(work_dir,'____Affine_0GenericAffine.mat')
    SyN = os.path.join(work_dir,'____SyN_0Warp.nii.gz')
    
    for input_file in input_file_list:
        output_file = os.path.join(os.path.dirname(input_file),
                               '______regd_'+os.path.basename(input_file)[2:])
        
        command_Affine =  ' '.join([
            antsApplyTransforms,'-d 3','-n','Linear','-v',
            '-i',input_file,
            '-r',template,
            '-t',Affine,
            '-o',output_file,
        ])
        command_SyN =  ' '.join([
            antsApplyTransforms,'-d 3','-n','Linear','-v',
            '-i',output_file,
            '-r',template,
            '-t',SyN,
            '-o',output_file,
        ])
        print(f'配准{input_file}')
        run_command(command_Affine,f'应用affine变换')
        run_command(command_SyN,f'应用SyN配准')

In [2]:
# 基本参数设置
MNI152 = 'icbm_avg_152_t1_tal_nlin_symmetric_VI.nii.gz'
antsRegistration= os.path.join('ANTs','antsRegistration.exe')
antsApplyTransforms= os.path.join('ANTs','antsApplyTransforms.exe')

def one_click(folder):
    # 先分出先后批次
    priority = ['__rsmp_T1CE.nii.gz', '__rsmp_T1.nii.gz', '__rsmp_T2.nii.gz', '__rsmp_T2Flair.nii.gz', 
                '__rsmp_fDWI_2.nii.gz', '__rsmp_ADC.nii.gz', '__rsmp_ADC_manual.nii.gz']
    aaaa = find_which_first(os.listdir(folder), priority)
    forerunner = aaaa[0]
    latecomer = aaaa[1]
    # 完整路径
    forerunner_full_path = os.path.join(folder,forerunner)
    latecomer_full_path = []
    for i in latecomer:
        latecomer_full_path.append(os.path.join(folder,i))  

    # 先对 forerunner 进行SyN配准，中间会生成 重采样文件(rsmp_xx) 和 三个转换矩阵文件
    SyN_regisitration(MNI152,forerunner_full_path)

    if len(latecomer_full_path)>0:
        # 所有互相配准后的文件应用变换到MNI152
        apply_all_transforms(MNI152,latecomer_full_path)

# one_click(r'G:\Aim_new\demo\X158510\20170504-Pre')

# 批量处理

In [3]:
# 找到对应文件夹里所有文件
root_path = r"G:\Aim_new"
max_depth = 4
directory_reader = DirectoryReader_dict(root_path, max_depth)
directories = directory_reader.get_files_dict()
print(len(directories))
directories

297


{'G:\\Aim_new\\REMBRANDT\\900-00-1961\\20050619-Pre': ['ADC.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  '__rsmp_ADC.nii.gz',
  '__rsmp_fDWI_2.nii.gz',
  '__rsmp_T1.nii.gz',
  '__rsmp_T1CE.nii.gz',
  '__rsmp_T2.nii.gz'],
 'G:\\Aim_new\\REMBRANDT\\900-00-5299\\20050322-Pre': ['ADC.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  'T2Flair.nii.gz',
  '__rsmp_ADC.nii.gz',
  '__rsmp_fDWI_2.nii.gz',
  '__rsmp_T1.nii.gz',
  '__rsmp_T1CE.nii.gz',
  '__rsmp_T2.nii.gz',
  '__rsmp_T2Flair.nii.gz'],
 'G:\\Aim_new\\REMBRANDT\\900-00-5303\\20050324-Pre': ['ADC.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  'T2Flair.nii.gz',
  '__rsmp_ADC.nii.gz',
  '__rsmp_fDWI_2.nii.gz',
  '__rsmp_T1.nii.gz',
  '__rsmp_T1CE.nii.gz',
  '__rsmp_T2.nii.gz',
  '__rsmp_T2Flair.nii.gz'],
 'G:\\Aim_new\\REMBRANDT\\900-00-5308\\20050424-Pre': ['ADC.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  'T2Flair.n

In [4]:
al_folders = list(directories.keys())

for i in tqdm(range(len(al_folders))):
    folder = al_folders[i]
    print(f"{'#'*80}\n{folder}")
    one_click(folder)

  0%|                                                                                          | 0/297 [00:00<?, ?it/s]

################################################################################
G:\Aim_new\REMBRANDT\900-00-1961\20050619-Pre
先配准: __rsmp_T1CE.nii.gz
后配准: ['__rsmp_T1.nii.gz', '__rsmp_ADC.nii.gz', '__rsmp_fDWI_2.nii.gz', '__rsmp_T2.nii.gz']
【拟生成文件】G:\Aim_new\REMBRANDT\900-00-1961\20050619-Pre\______regd_rsmp_T1CE.nii.gz
【Affine放射变换】命令执行完毕,耗时6.86秒
【应用affine变换】命令执行完毕,耗时3.88秒
【SyN配准变换】命令执行完毕,耗时39.85秒
【应用syn配准】命令执行完毕,耗时3.68秒
配准G:\Aim_new\REMBRANDT\900-00-1961\20050619-Pre\__rsmp_T1.nii.gz
【应用affine变换】命令执行完毕,耗时3.27秒
【应用SyN配准】命令执行完毕,耗时3.28秒
配准G:\Aim_new\REMBRANDT\900-00-1961\20050619-Pre\__rsmp_ADC.nii.gz
【应用affine变换】命令执行完毕,耗时1.96秒
【应用SyN配准】命令执行完毕,耗时2.24秒
配准G:\Aim_new\REMBRANDT\900-00-1961\20050619-Pre\__rsmp_fDWI_2.nii.gz
【应用affine变换】命令执行完毕,耗时2.68秒
【应用SyN配准】命令执行完毕,耗时3.01秒
配准G:\Aim_new\REMBRANDT\900-00-1961\20050619-Pre\__rsmp_T2.nii.gz
【应用affine变换】命令执行完毕,耗时3.07秒
【应用SyN配准】

  0%|▎                                                                               | 1/297 [01:16<6:19:32, 76.93s/it]

命令执行完毕,耗时3.15秒
################################################################################
G:\Aim_new\REMBRANDT\900-00-5299\20050322-Pre
先配准: __rsmp_T1CE.nii.gz
后配准: ['__rsmp_T1.nii.gz', '__rsmp_ADC.nii.gz', '__rsmp_fDWI_2.nii.gz', '__rsmp_T2Flair.nii.gz', '__rsmp_T2.nii.gz']
【拟生成文件】G:\Aim_new\REMBRANDT\900-00-5299\20050322-Pre\______regd_rsmp_T1CE.nii.gz
【Affine放射变换】命令执行完毕,耗时6.35秒
【应用affine变换】命令执行完毕,耗时3.03秒
【SyN配准变换】命令执行完毕,耗时37.59秒
【应用syn配准】命令执行完毕,耗时3.87秒
配准G:\Aim_new\REMBRANDT\900-00-5299\20050322-Pre\__rsmp_T1.nii.gz
【应用affine变换】命令执行完毕,耗时3.46秒
【应用SyN配准】命令执行完毕,耗时3.15秒
配准G:\Aim_new\REMBRANDT\900-00-5299\20050322-Pre\__rsmp_ADC.nii.gz
【应用affine变换】命令执行完毕,耗时1.63秒
【应用SyN配准】命令执行完毕,耗时2.11秒
配准G:\Aim_new\REMBRANDT\900-00-5299\20050322-Pre\__rsmp_fDWI_2.nii.gz
【应用affine变换】命令执行完毕,耗时3.11秒
【应用SyN配准】命令执行完毕,耗时3.42秒
配准G:\Aim_new\REMBRANDT\900-00-5299\20050322-Pre\__rsmp_T2Flair.nii.gz
【应用affine变换】命令执行完毕,耗时3.25秒
【应用SyN配准】命令执行完毕,耗时3.08秒
配准G:\Aim_new\REMBRANDT\900-00-5299\20050322-Pre\__rsmp_T2.ni

  1%|▌                                                                               | 2/297 [02:37<6:28:18, 78.98s/it]

...

命令执行完毕,耗时3.74秒
################################################################################
G:\Aim_new\TCGA_LGG\TCGA-HT-A617\19970722-Pre
先配准: __rsmp_T1CE.nii.gz
后配准: ['__rsmp_T1.nii.gz', '__rsmp_T2.nii.gz']
【拟生成文件】G:\Aim_new\TCGA_LGG\TCGA-HT-A617\19970722-Pre\______regd_rsmp_T1CE.nii.gz
【Affine放射变换】命令执行完毕,耗时4.67秒
【应用affine变换】命令执行完毕,耗时3.00秒
【SyN配准变换】命令执行完毕,耗时38.52秒
【应用syn配准】命令执行完毕,耗时3.83秒
配准G:\Aim_new\TCGA_LGG\TCGA-HT-A617\19970722-Pre\__rsmp_T1.nii.gz
【应用affine变换】命令执行完毕,耗时3.58秒
【应用SyN配准】命令执行完毕,耗时3.75秒
配准G:\Aim_new\TCGA_LGG\TCGA-HT-A617\19970722-Pre\__rsmp_T2.nii.gz
【应用affine变换】命令执行完毕,耗时2.94秒
【应用SyN配准】

100%|█████████████████████████████████████████████████████████████████████████████▋| 296/297 [6:27:00<01:09, 69.62s/it]

命令执行完毕,耗时3.58秒
################################################################################
G:\Aim_new\TCGA_LGG\TCGA-HT-A61A\20000127-Pre
先配准: __rsmp_T1CE.nii.gz
后配准: ['__rsmp_T2Flair.nii.gz', '__rsmp_T1.nii.gz', '__rsmp_T2.nii.gz']
【拟生成文件】G:\Aim_new\TCGA_LGG\TCGA-HT-A61A\20000127-Pre\______regd_rsmp_T1CE.nii.gz
【Affine放射变换】命令执行完毕,耗时4.61秒
【应用affine变换】命令执行完毕,耗时3.02秒
【SyN配准变换】命令执行完毕,耗时42.84秒
【应用syn配准】命令执行完毕,耗时3.58秒
配准G:\Aim_new\TCGA_LGG\TCGA-HT-A61A\20000127-Pre\__rsmp_T2Flair.nii.gz
【应用affine变换】命令执行完毕,耗时3.53秒
【应用SyN配准】命令执行完毕,耗时3.76秒
配准G:\Aim_new\TCGA_LGG\TCGA-HT-A61A\20000127-Pre\__rsmp_T1.nii.gz
【应用affine变换】命令执行完毕,耗时3.27秒
【应用SyN配准】命令执行完毕,耗时3.66秒
配准G:\Aim_new\TCGA_LGG\TCGA-HT-A61A\20000127-Pre\__rsmp_T2.nii.gz
【应用affine变换】命令执行完毕,耗时3.57秒
【应用SyN配准】

100%|██████████████████████████████████████████████████████████████████████████████| 297/297 [6:28:16<00:00, 78.44s/it]

命令执行完毕,耗时3.71秒
